#### ExPred
This notebook can be used generate parts of version 1, as well as version 2 of the prototypes for presentations of the attribution technique ExPred
Required Input: Path to fever jsonl data set. (e.g. test.jsonl)

In [1]:
# imports
import json
import torch
from src.expred import (seeding, ExpredInput,
                        BertTokenizerWithSpans, ExpredConfig, Expred)
from src.expred.models import (prepare_for_cl, train_evidence_classifier,
                               train_mtl_token_identifier)
from transformers import BertTokenizer
import ssl

In [ ]:
# function to transform the input to the way the expred accepts
def formatData(path_to_file, path_to_fever_docs):
    data = []
    with open(path_to_file, 'r') as json_file:
        json_list = list(json_file)

    for json_str in json_list:
        result = json.loads(json_str)
        label = result.get('classification')
        query = result.get('query').split()
        for docs_id in result.get('docids'):
            with open(f'{path_to_fever_docs}/{docs_id}', 'r') as json_file_2:
                documents = list(json_file_2)
                contexts = []
                for doc in documents:
                    contexts.append(doc.split())
            data.append([label,query,contexts])
    return data

In [ ]:
# function with code from expred showcase
def getExPredResult(data): 
    ssl._create_default_https_context = ssl._create_unverified_context

    expred_config = ExpredConfig(
        pretrained_dataset_name='fever',
        base_dataset_name='fever',
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
        load_from_pretrained=True)

    # seeding
    seeding(1234)

    # Initialize tokenizer
    tokenizer = BertTokenizerWithSpans.from_pretrained('bert-base-uncased')

    # create the model
    expred = Expred.from_pretrained(expred_config)
    expred.eval()

    for data_point in data:
        # print query and data to paste into figma template
        print('Query: ', ' '.join(data_point[1]))
        # flatten whole data 
        print('Whole data: ', ' '.join([item for sublist in data_point[2] for item in sublist]))
        expred_input = ExpredInput(
            queries = [data_point[1]],
            docs = [data_point[2]],
            labels = [data_point[0]],
            config = expred_config,
            ann_ids = ['spontan_1'],
            span_tokenizer = tokenizer)
        # don't forget to preprocess
        expred_input.preprocess()

        # the output is in the form of a dict:
        expred_output = expred(expred_input)

        # functions from ExPred
        expred_preds_output = expred_input.get_decoded_exp_preds(expred_output)[0]
        # optionally remove dots
        # expred_preds_output = [i for i in expred_preds_output if i!=('.')]
        print('Influencial tokens: ', ' '.join(expred_preds_output))
        print('Classification: ',expred_input.get_decoded_cls_preds(expred_output)[0])

In [ ]:
# After running this function the output can be used to copy/paste into ExPred Prototypes in Figma
# Note that the highlighting was done manually

data = formatData('replace with path to jsonl', 'replace with path to fever docs')
getExPredResult(data)

FileNotFoundError: [Errno 2] No such file or directory: 'replace with path to jsonl'